
# <font color='056938'> **Librerias** </font>

In [55]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import logging
from tqdm import tqdm
import os
import sys



# <font color='056938'> **Conexion con Drive** </font>

In [56]:
# Conexion con Drive
from google.colab import drive
drive.mount('/content/drive')

# Cambio de directorio de Trabajo

path = '/content/drive/MyDrive/Mod2/ANALITICA3'
os.chdir(path)
sys.path.append(path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# <font color='056938'> **Cargar base de datos** </font>

In [57]:
conn=sql.connect('/content/drive/MyDrive/Mod2/ANALITICA3/data/db_movies')
cur=conn.cursor() ###para funciones que ejecutan sql en base de datos

# <font color='056938'> **Configuración del log** </font>

In [58]:
logging.basicConfig(
    filename='/content/drive/MyDrive/Mod2/ANALITICA3/logs/script_log.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [59]:
# Cargar la tabla completa
tabla_completa = pd.read_sql_query("SELECT * FROM full_ratings", conn)

# Verificación de la tabla cargada
tabla_completa.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25978 entries, 0 to 25977
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          25978 non-null  int64  
 1   movie_id         25978 non-null  int64  
 2   movie_title      25978 non-null  object 
 3   movie_genres     25978 non-null  object 
 4   movie_rating     25978 non-null  float64
 5   movie_timestamp  25978 non-null  int64  
 6   movie_year       25978 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 1.4+ MB


In [60]:
# Escalar el año para que esté en el mismo rango
sc = StandardScaler()
tabla_completa[["year_sc"]] = sc.fit_transform(tabla_completa[['movie_year']])



**Separacion de generos**

In [61]:
# Separar los géneros y expandirlos en múltiples filas
df_genres = tabla_completa.assign(movie_genres=tabla_completa['movie_genres'].str.split('|')).explode('movie_genres')
df_genres = df_genres.rename(columns={'movie_genres': 'genre'})


In [62]:
# Eliminar las columnas que no se van a usar
movie_dum1 = df_genres.drop(columns=['user_id', 'movie_rating', 'movie_timestamp', 'movie_year', 'movie_title'])


 **Dumización de géneros**

In [63]:
movie_dum1['genre'].nunique()
col_dum = ['genre']
movie_dum2 = pd.get_dummies(movie_dum1, columns=col_dum)

In [64]:
# Unificar las películas por 'movie_id'
movie_unificado = movie_dum2.groupby('movie_id', as_index=False).max()


In [65]:
def Tabla_procesada(conn):
    # Cargar la tabla
    tabla_completa = pd.read_sql_query("SELECT * FROM full_ratings", conn)

    # Escalar el año
    sc = StandardScaler()
    tabla_completa[["year_sc"]] = sc.fit_transform(tabla_completa[['movie_year']])

    # Separar géneros y expandir
    df_genres = tabla_completa.assign(movie_genres=tabla_completa['movie_genres'].str.split('|')).explode('movie_genres')
    df_genres = df_genres.rename(columns={'movie_genres': 'genre'})

    # Eliminar columnas innecesarias
    movie_dum1 = df_genres.drop(columns=['user_id', 'movie_rating', 'movie_timestamp', 'movie_year', 'movie_title'])

    # Dumificación
    movie_dum2 = pd.get_dummies(movie_dum1, columns=['genre'])

    # Unificar por movie_id
    movie_unificado = movie_dum2.groupby('movie_id', as_index=False).max()

    # También devolver tabla original para nombres de películas
    return movie_unificado, tabla_completa

# <font color='056938'> **Modelo KNN basado en contenido** </font>

In [66]:
model = NearestNeighbors(n_neighbors=10, metric='cosine')
model.fit(movie_unificado)


NearestNeighbors(metric='cosine', n_neighbors=10)

In [67]:
# Función interactiva para obtener recomendaciones
def MovieRecommender(movie_name=list(tabla_completa['movie_title'].value_counts().index)):
    movie_list_name = []
    movie_id = tabla_completa[tabla_completa['movie_title'] == movie_name].index
    movie_id = movie_id[0]

    for newid in idlist[movie_id]:
        movie_list_name.append(tabla_completa.loc[newid].movie_title)

    return movie_list_name


In [68]:
import sys
sys.path.append('/content/drive/MyDrive/Mod2/ANALITICA3/Trabajo Marketing')

from preprocesamiento import Tabla_procesada

def preprocesar(conn=None, cur=None):
    movie_dum2, movie = Tabla_procesada(conn)
    return movie_dum2, movie, conn, cur


In [69]:
# Función para procesar y guardar recomendaciones
def recomendar(user_id, conn=None, cur=None, movie_dum2=None, movie=None):
    ratings = pd.read_sql('SELECT * FROM ratings_final WHERE user_id=:user', conn, params={'user': user_id})

    if ratings.empty:
        logging.warning(f"No hay ratings para el usuario {user_id}")
        return pd.DataFrame()

    l_movies_r = ratings['movie_id'].to_numpy()
    movie_dum2[['movie_id', 'movie_title']] = movie[['movie_id', 'movie_title']]
    movies_r = movie_dum2[movie_dum2['movie_id'].isin(l_movies_r)]
    movies_r = movies_r.drop(columns=['movie_id', 'movie_title'])
    movies_r["indice"] = 1
    centroide = movies_r.groupby("indice").mean()

    movies_nr = movie_dum2[~movie_dum2['movie_id'].isin(l_movies_r)]

    if movies_nr.empty:
        logging.warning(f"Usuario {user_id} ya ha visto todas las películas. No se puede recomendar.")
        return pd.DataFrame()

    movies_nr = movies_nr.drop(columns=['movie_id', 'movie_title'])

    model = NearestNeighbors(n_neighbors=min(11, len(movies_nr)), metric='cosine')
    model.fit(movies_nr)
    dist, idlist = model.kneighbors(centroide)

    ids = idlist[0]
    recomend_b = movie.loc[ids][['movie_title', 'movie_id']]
    return recomend_b


In [70]:
# Función para procesar y guardar recomendaciones
def main(list_user):
    conn = sql.connect('/content/drive/MyDrive/Mod2/ANALITICA3/data/db_movies')
    cur = conn.cursor()

    recomendaciones_todos = pd.DataFrame()
    movie_dum2, movie, conn, cur = preprocesar(conn, cur)

    for user_id in tqdm(list_user):
        recomendaciones = recomendar(user_id, conn, cur, movie_dum2, movie)
        recomendaciones["user_id"] = user_id
        recomendaciones.reset_index(inplace=True, drop=True)

        recomendaciones_todos = pd.concat([recomendaciones_todos, recomendaciones])

    # Guardar las recomendaciones generadas
    recomendaciones_todos.to_excel('/content/drive/MyDrive/Mod2/ANALITICA3/Trabajo Marketing/recomendaciones.xlsx')
    recomendaciones_todos.to_csv('/content/drive/MyDrive/Mod2/ANALITICA3/Trabajo Marketing/recomendaciones.csv')

# Ejecutar el código principal
if __name__ == "__main__":
    list_user = [213, 381, 580]
    main(list_user)

100%|██████████| 3/3 [00:00<00:00, 45.34it/s]
